## Doc2Vec 시도

#### 데이터 전처리

In [1]:
import pandas as pd
from konlpy.tag import Okt
from gensim.models.doc2vec import TaggedDocument
from tqdm import tqdm

In [2]:
tagger = Okt()

In [3]:
df = pd.read_excel("./context.xlsx", engine="openpyxl", header=0)

In [4]:
df.head()

,Unnamed: 0,0,1
0,0,나는 공부를 할 때 반복을 중요시하였다,0
1,1,특히 수학 공부를 할 때에 오답이나 헷갈렸던 문제는 해당 공식과 함께 노트에 따로 ...,0
2,2,노트 정리 후에는 답을 스스로 찾을 때까지 여러 번 반복해 문제를 풀고 풀 때마다 ...,0
3,3,그래서 수학은 문제 풀이 시간과 실수 모두 줄이게 되 었다,0
4,4,영어는 원서를 읽으며 문장 구조와 어휘를 분석하며 읽는 습관을 들였다,0


In [5]:
tagged_corpus_list = []
for index, row in tqdm(df.iterrows(), total=len(df)):
    text = row.iloc[1]
    tag = row.iloc[0]
    tagged_corpus_list.append(TaggedDocument(tags=[tag], words=tagger.morphs(text)))

100%|█████████████████████████████████████████████████████████████████████████████| 1869/1869 [00:12<00:00, 152.32it/s]


In [6]:
tagged_corpus_list[0]

TaggedDocument(words=['나', '는', '공부', '를', '할', '때', '반복', '을', '중요시', '하였다'], tags=[0])

In [7]:
from gensim.models import doc2vec

In [8]:
model = doc2vec.Doc2Vec(vector_size=360, alpha=0.025, min_alpha=0.025, workers=8, window=8)

In [9]:
model.build_vocab(tagged_corpus_list)

In [10]:
model.train(corpus_iterable=tagged_corpus_list, total_examples=model.corpus_count, epochs=50)

In [11]:
model.save('coverLetter.doc2vec')

In [12]:
similar_doc = model.dv.most_similar(15)
print(similar_doc)

[(513, 0.5717862844467163), (464, 0.5653696656227112), (1092, 0.5646870732307434), (690, 0.5601072907447815), (812, 0.553207278251648), (35, 0.5440871715545654), (922, 0.5428146123886108), (179, 0.5208689570426941), (440, 0.5197383165359497), (666, 0.5153456330299377)]


In [46]:
def get_similar_Centence(_index):
    print("원본:", df.iloc[_index, 1])
    print('-'*20)
    similar_doc = model.dv.most_similar(_index)
    index_list = [similar_doc[i][0] for i in range(0, 5)]
    for i in range(len(index_list)):
        j = index_list[i]
        print("{0} : {1} ({2:0.1f}%)".format(i+1, df.iloc[j, 1], similar_doc[i][1]*100))
        print('-'*20)

In [47]:
get_similar_Centence(15)

원본: 또 중국어 문화에 대한 수업과 동아리 활동으로 문화 차이로 인한 다문화 가정 내 갈등 상황에 대해서도 조사해보고 다문화 가정 관련 문제로 인한 법 문제를 모의로 해결하는 법학 동아리도 만들어 활동해보고 싶다
--------------------
1 : 또 지구 온난화 해결 동아리와 경제 동아리에서 환경과 경제의 상관관계에 대해서도 연구해보고 싶다 (57.2%)
--------------------
2 : 또 학생 주도 소그룹 활동에 참여해 친구들과 법에 대한 심화 지식도 쌓고 싶다 (56.5%)
--------------------
3 : 먼저 전 세계 빈곤 문제를 해결하는 내 꿈을 위해 빈곤 문제의 현황을 조사해 해결방안을 고민해보았다 (56.5%)
--------------------
4 : 또 하브루타 교육동아리를 개설해 문제 상황에 따른 교육자의 대처 방안을 친구들과 함께 고민해보고 싶다 (56.0%)
--------------------
5 : 또 조별과제로 일본군 위안부 문제에 대해 조사하였다 (55.3%)
--------------------


In [48]:
get_similar_Centence(69)

원본: 영어토론동아리에서는 친구들과 다양한 분야의 주제를 선정하여 넓은 시야를 가지고 영어토론을 할 수 있게 되었다
--------------------
1 : 또 친구들과 다양한 사회 주제를 영어로 토론하며 통찰력을 키웠다 (73.2%)
--------------------
2 : 나는 친구들에게 나의 재능을 나누며 나눔을 실천하였다 (70.0%)
--------------------
3 : 또 3학년 때는 영어토론 동아리에 참여해 친구들과 다양한 주제에 대해 토론했다 (68.5%)
--------------------
4 : 또 훗날 해외에서의 의료봉사를 위한 회화 실력을 기르고자 여러 사회 이슈에 대해 친구들과 토론도 해보았다 (66.8%)
--------------------
5 : 입학하면 GLTC (66.3%)
--------------------


In [49]:
get_similar_Centence(125)

원본: 2학년 때 저소득층 아이들 교육 봉사를 했다
--------------------
1 : 3년간 다문화 가정 아이들을 위해 중국어 교육 봉사를 했다 (82.4%)
--------------------
2 : 3학년 때는 애니메이션 동아리에서 활동했다 (81.7%)
--------------------
3 : 입학하면 GLTC (81.4%)
--------------------
4 : 나는 친구들에게 나의 재능을 나누며 나눔을 실천하였다 (78.5%)
--------------------
5 : 나는 2학년 때 경로당 어르신이 가족과 함께 즐기실 수 있도록 할리갈리 컵스 게임을 가르쳐드리는 봉사를 했다 (77.6%)
--------------------
